## Setup

Downloads the FHIR Validator from https://github.com/hapifhir/org.hl7.fhir.core/releases/latest/download/validator_cli.jar

For more details on the FHIR Validator see [Using the FHIR Validator](https://confluence.hl7.org/spaces/FHIR/pages/35718580/Using+the+FHIR+Validator)

Downloads the NW Genomics validation package

In [43]:
from tqdm import tqdm
import requests

print("Downloading the FHIR Validator...")

url = "https://github.com/hapifhir/org.hl7.fhir.core/releases/latest/download/validator_cli.jar"
response = requests.get(url, stream=True)

with open("validator_cli.jar", "wb") as handle:
    for data in tqdm(response.iter_content(chunk_size=1024), unit="kB"):
        handle.write(data)

print("Downloading the North West Genomics validation package...")
url = "https://nw-gmsa.github.io/package.tgz"
response = requests.get(url, stream=True)

with open("package.tgz", "wb") as handle:
    for data in tqdm(response.iter_content(chunk_size=1024), unit="kB"):
        handle.write(data)


Now run a validation using a FHIR Message Bundle.

The example is an O21 Order, so we also need to help the validator by specifying the profile of the focus resource which in this case is a ServiceRequest which should adhere to https://nw-gmsa.github.io/StructureDefinition-ServiceRequest.html (profile url https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest)

A default output is returned from the validator.

Using command line this is:

>  java -jar validator_cli.jar Input/FHIR/O21/Liverpool_O21.json -version 4.0.1 -ig package.tgz -bundle ServiceRequest:0 https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest -tx n/a

In [44]:
import subprocess

filename = "Liverpool_O21.json"
folder = 'FHIR/O21/'

subprocess.run(["java", "-jar", "validator_cli.jar", "Input/"+folder + filename , "-version", "4.0.1", "-ig", "package.tgz", "-bundle", "ServiceRequest:0", "https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest", "-tx", "n/a"])

CompletedProcess(args=['java', '-jar', 'validator_cli.jar', 'Input/FHIR/O21/Liverpool_O21.json', '-version', '4.0.1', '-ig', 'package.tgz', '-bundle', 'ServiceRequest:0', 'https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest', '-tx', 'n/a'], returncode=0)

The output can contain a mix of issues which may be difficult to understand. If we get a large number of issues, we may want to prioritise `error` first and write a test report. So we need to output the results in a machine readable format



In [45]:
import subprocess

subprocess.run(["java", "-jar", "validator_cli.jar", "Input/"+folder + filename, "-version", "4.0.1", "-ig", "package.tgz", "-bundle", "ServiceRequest:0", "https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest", "-tx", "n/a", "-output", "Results/"+folder + filename + "-OperationOutcome.json", "-output-style","json"])

CompletedProcess(args=['java', '-jar', 'validator_cli.jar', 'Input/FHIR/O21/Liverpool_O21.json', '-version', '4.0.1', '-ig', 'package.tgz', '-bundle', 'ServiceRequest:0', 'https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest', '-tx', 'n/a', '-output', 'Results/FHIR/O21/Liverpool_O21.json-OperationOutcome.json', '-output-style', 'json'], returncode=0)

In [46]:
import json
import pandas as pd

def details(item):
    return item['text']

with open( "Results/"+folder + filename + "-OperationOutcome.json", 'rb') as f:
    s = f.read().decode('utf-8')
    operationOutcome = json.loads(s)

df = pd.DataFrame(operationOutcome['issue'])

df['details'] = df['details'].apply(details)
df.drop(columns=['extension'], inplace=True)
df.sort_values(by=['severity'], inplace=True)


df = df[~df['details'].str.contains('ValueSet/mimetypes')]
df = df[~df['details'].str.contains('failed: dom-6')]
df = df[~df['details'].str.contains('bcp:13')]

df

,severity,code,details,expression
0,error,invalid,"The property identifier must be an Object, not...",[Bundle.entry[2].resource.identifier[0].assign...
114,error,structure,"Observation.code: minimum required = 1, but on...",[Bundle.entry[20].resource/*Observation/735aae...
2,error,structure,MessageHeader.source.endpoint: minimum require...,[Bundle.entry[0].resource/*MessageHeader/64001...
142,error,structure,Unable to find a profile match for urn:uuid:dc...,[Bundle.entry[1].resource.subject]
4,error,structure,"ServiceRequest.status: minimum required = 1, b...",[Bundle.entry[1].resource/*ServiceRequest/ee6a...
...,...,...,...,...
82,warning,invalid,"Best Practice Recommendation: In general, all ...",[Bundle.entry[15].resource/*Observation/OBX-7-...
81,warning,invalid,"Best Practice Recommendation: In general, all ...",[Bundle.entry[15].resource/*Observation/OBX-7-...
80,warning,invariant,Constraint failed: dom-6: 'A resource should h...,[Bundle.entry[14].resource/*Observation/OBX-23...
88,warning,invalid,"Best Practice Recommendation: In general, all ...",[Bundle.entry[16].resource/*Observation/OBX-4-...


In [54]:
import jinja2
import weasyprint
from datetime import date

html_string = df.to_html()

html = (
    jinja2.Environment(loader=jinja2.FileSystemLoader(searchpath=""))
    .get_template(name="Results/report_template.html")
    .render(
        date=date.today().strftime("%d, %b %Y"),
        table=html_string
    )
)

with open("Results/"+folder + filename + "-report.html", "w") as out_html_file_handle:
    out_html_file_handle.write(html)

with open("Results/"+folder + filename + "-report.html", "r") as html_file_handle:
    pdf = weasyprint.HTML(html_file_handle).write_pdf()


open("Results/"+folder + filename + '-report.pdf', 'wb').write(pdf)



31072